# Specifying data with Batch Definitions

Before running this notebook, make sure you're gone through "Authoring Expectation Suites", which connects to data.

## Basic setup

Imports

In [9]:
import great_expectations as gx
from great_expectations.core.validation_definition import ValidationDefinition
from great_expectations.datasource.fluent.interfaces import Datasource
from great_expectations.exceptions import DataContextError

from constants import (
    DATASOURCE_NAME,
    ASSET_NAME,
    SUITE_NAME,
    BATCH_DEFINITION_NAME_PARTIONED_DESC,
    BATCH_DEFINITION_NAME_PARTIONED,
)

### Get our Datasource and DataAsset

In [8]:
context = gx.get_context(mode="file")

datasource = context.get_datasource(DATASOURCE_NAME)
assert isinstance(datasource, Datasource)
data_asset = datasource.get_asset(asset_name=ASSET_NAME)
suite = context.suites.get(SUITE_NAME)

### Create monthly batch definitions

We previously created "whole table" Batch Definitions.
Now, we'll create monthly batch definitions.
When checkpoints use these, they will run expectation suites against the last batch defined by the Batch Definition.

In [ ]:
try:
    batch_def = data_asset.add_batch_definition_monthly(
        BATCH_DEFINITION_NAME_PARTIONED,
        column="tpep_pickup_datetime"
    )
    batch_def_desc = data_asset.add_batch_definition_monthly(
        BATCH_DEFINITION_NAME_PARTIONED_DESC,
        column="tpep_pickup_datetime",
        sort_ascending=False
    )
    print("Created BatchDefinitions")
except DataContextError:
    batch_def = next(bd for bd in data_asset.batch_definitions if bd.name == BATCH_DEFINITION_NAME_PARTIONED)
    batch_def_desc = next(bd for bd in data_asset.batch_definitions if bd.name == BATCH_DEFINITION_NAME_PARTIONED_DESC)
    print("Entities already exist")


### Run the Validation Definition for a monthly Batch Definition

In [ ]:
validation_definition = ValidationDefinition(name="ephemeral validation definition", data=batch_def_desc, suite=suite)

results = validation_definition.run()
print(results.describe())

### Specifying the Batch from a Batch Definition

We can specify the batch by providing batch_parameters. A monthly Batch Definition needs parameters for `month` and `year`. These are passed into `ValidationDefinition::run` as a dictionary named `batch_parameters`.

In [ ]:
results = validation_definition.run(batch_parameters={"month": 2020, "month": 5})
print(results.describe())